# Examples on the Front Page

## Expected Loss

In [1]:
import os
import sys


sys.path.append(os.path.abspath('../'))

In [ ]:
import quactuary as qa
import quactuary.book as book

from datetime import date
from quactuary.backend import set_backend
from quactuary.book import (
    ExposureBase, LOB, PolicyTerms, Inforce, Portfolio)
from quactuary.distributions.frequency import Poisson, NegativeBinomial
from quactuary.distributions.severity import Pareto, Lognormal
from quactuary.pricing import PricingModel


# Workers’ Comp Bucket
wc_policy = PolicyTerms(
    effective_date=date(2026, 1, 1),
    expiration_date=date(2027, 1, 1),
    lob=LOB.WC,
    exposure_base=book.PAYROLL,
    exposure_amount=100_000_000,
    retention_type="deductible",
    per_occ_retention=500_000,
    coverage="occ"
)

# General Liability Bucket
glpl_policy = PolicyTerms(
    effective_date=date(2026, 1, 1),
    expiration_date=date(2027, 1, 1),
    lob=LOB.GLPL,
    exposure_base=book.SALES,
    exposure_amount=10_000_000_000,
    retention_type="deductible",
    per_occ_retention=1_000_000,
    coverage="occ"
)

# Frequency-Severity Distributions
wc_freq = Poisson(100)
wc_sev = Pareto(1, 0, 40_000)

glpl_freq = NegativeBinomial(50, 0.5)
glpl_sev = Lognormal(2, 0, 100_000)

# Book of Business
wc_inforce = Inforce(
    n_policies=1000,
    terms=wc_policy,
    frequency=wc_freq,
    severity=wc_sev,
    name = "WC 2026 Bucket"
)

glpl_inforce = Inforce(
    n_policies=700,
    terms=glpl_policy,
    frequency=glpl_freq,
    severity=glpl_sev,
    name = "GLPL 2026 Bucket"
)

portfolio = wc_inforce + glpl_inforce

pm = PricingModel(portfolio)

# Test using Classical Monte Carlo
set_backend("classical")
classical_result = pm.simulate(n_sims=1_000)
classical_mean = classical_result.estimates["mean"]
print(f"Classical portfolio expected loss: {classical_mean}")

Classical portfolio expected loss: 115557736308.56332


## Risk Measures

In [4]:
from quactuary.backend import use_backend
from quactuary.distributions.frequency import Geometric
from quactuary.distributions.severity import ContinuousUniformSeverity


# Commercial Auto Bucket
cauto_policy = PolicyTerms(
    effective_date=date(2026, 1, 1),
    expiration_date=date(2027, 1, 1),
    lob=LOB.CAuto,
    exposure_base=book.VEHICLES,
    exposure_amount=50,
    retention_type="deductible",
    per_occ_retention=100_000,
    coverage="occ"
)

# Frequency-Severity Distributions
cauto_freq = Geometric(1/8)
cauto_sev = ContinuousUniformSeverity(5_000, 95_000)

# Commercial Auto Inforce
cauto_inforce = Inforce(
    n_policies=400,
    terms=cauto_policy,
    frequency=cauto_freq,
    severity=cauto_sev,
    name = "CAuto 2026 Bucket"
)

# Add to Existing Portfolio
portfolio += cauto_inforce
pm2 = PricingModel(portfolio)

# Test using Classical Monte Carlo
with use_backend("classical", num_simulations=1_000):
    classical_result = pm2.simulate(tail_alpha=0.05, n_sims=1_000)
    classical_VaR = classical_result.estimates["VaR"]
    classical_TVaR = classical_result.estimates["TVaR"]
    print(f"Classical portfolio VaR: {classical_VaR}")
    print(f"Classical portfolio TVaR: {classical_TVaR}")

Classical portfolio VaR: 166575766183.6848
Classical portfolio TVaR: 5490672479515.086
